In [1]:
import json
import yaml
import numpy as np
import shutil
from pathlib import Path
import pandas as pd
from mouselab.mouselab import MouselabEnv
from mouselab.env_utils import get_ground_truths_from_json
import dill as pickle
from collections import Counter
from costometer.utils.analysis_utils import AnalysisObject

# Creating data, modifying code

In [2]:
discount_factors = [.10, .20, .30, .40, .50,.75]#,.85]#.9,.95,.975]
alphas = [0.0,.25,.5,.75,.9]

Put this in `cluster/src/cluster_utils.py`

Generate cluster submission commands to run to get Q values

In [3]:
print(f"for gamma in 1 {' '.join([str(discount_factor) for discount_factor in discount_factors])};\n"
      f"    do for alpha in 1 {' '.join([str(alpha) for alpha in alphas])};\n"
      f"        do condor_submit_bid 2 submission_scripts/MPI-IS/03_Get_Q_Values.sub experiment_setting=high_increasing cost_function=back_dist_depth_eff_forw param_file=back_dist_depth_eff_forw alpha=$alpha gamma=$gamma;\n"
      f"done; done;")

for gamma in 1 0.1 0.2 0.3 0.4 0.5 0.75;
    do for alpha in 1 0.0 0.25 0.5 0.75 0.9;
        do condor_submit_bid 2 submission_scripts/MPI-IS/03_Get_Q_Values.sub experiment_setting=high_increasing cost_function=back_dist_depth_eff_forw param_file=back_dist_depth_eff_forw alpha=$alpha gamma=$gamma;
done; done;


Generate cluster submission commands to infer likelihood for methods_main:

In [ ]:
print(f"for gamma in 1 {' '.join([str(discount_factor) for discount_factor in discount_factors])};\n"
      f"    do for alpha in 1 {' '.join([str(alpha) for alpha in alphas])};\n"
      f"        do condor_submit_bid 2 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main cost_function=back_dist_depth_eff_forw param_file=dist_depth_eff_forw alpha=$alpha gamma=$gamma;\n"
      f"done; done;")

Combine data:

In [ ]:
print(f"for experiment in methods_main {' '.join(['methods_main' + str(discount_factor) for discount_factor in discount_factors])};\n"
      f"    do for alpha in 1 {' '.join([str(alpha) for alpha in alphas])};\n"
      f"        do condor_submit_bid 2 submission_scripts/MPI-IS/05_Combine_Human.sub experiment=$experiment cost_function=dist_depth_eff_forw alpha=$alpha;\n"
      f"done; done;")

MAP:

In [ ]:
print(f"for experiment in methods_main {' '.join(['methods_main' + str(discount_factor) for discount_factor in discount_factors])};\n"
      f"    do for alpha in 1 {' '.join([str(alpha) for alpha in alphas])};\n"
      f"        do condor_submit_bid 2 submission_scripts/MPI-IS/M_01_Get_MAP_File_by_PID.sub pid_file=methods_main pid_file=methods_main experiment=$experiment cost_function=dist_forw_eff alpha=$alpha;\n"
      f"done; done;")

In [ ]:
with open(Path("../../../analysis/methods/static/inputs/yamls/NoDepth.yaml"), "rb") as f:
    analysis_setting = yaml.safe_load(f)
  
for experiment_suffix in discount_factors:
    params = [f"{experiment_suffix:.3f}{alpha_suffix}" for alpha_suffix in [""] + [f"_{alpha:.2f}" for alpha in alphas]]

    analysis_setting["params"] = params
    
    with open(Path(f"../../../analysis/methods/static/inputs/yamls/DiscountFactor{experiment_suffix}.yaml"), "w") as f:
        yaml.dump(analysis_setting, f)

In [ ]:
experiment_val_dict = {} 
alpha_val_dict = {}
for experiment_val, experiment_suffix in zip([1]+ discount_factors, [""] + discount_factors):
    for alpha_val, alpha_suffix in zip([1] + alphas, [""] + [f"_{alpha:.2f}" for alpha in alphas]):
        if experiment_suffix == "":
            alpha_val_dict[f"{experiment_suffix}{alpha_suffix}"] = alpha_val
            experiment_val_dict[f"{experiment_suffix}{alpha_suffix}"] = experiment_val
        else:
            alpha_val_dict[f"{experiment_suffix:.3f}{alpha_suffix}"] = alpha_val
            experiment_val_dict[f"{experiment_suffix:.3f}{alpha_suffix}"] = experiment_val

Transfer over:

```
rsync -aPzr --include "methods_main*/*/*mle_and_map*" --include "*/" --exclude "*" vfelso@login.cluster.is.localnet:/work/vfelso/planning-depth-differences/data/processed/ data/processed
```

# Fit of main dataset by discount factor + power utility function model instead of depth in winning model

Distance, Effort, Depth and Forward Search Bonus              40490.81774

In [ ]:
# optimization_dfs = []

# for experiment_suffix in [""] + discount_factors:
#     analysis_obj = AnalysisObject(f"DiscountFactor{experiment_suffix}", irl_path=Path("../../..").resolve(), experiment_subdirectory="methods/static")

#     optimization_data = analysis_obj.query_optimization_data(include_null=False)
#     optimization_data=optimization_data[optimization_data["Model Name"]=="Distance and Effort Costs with Forward Search Bonus"]

#     optimization_dfs.append(optimization_data)

# optimization_dfs = pd.concat(optimization_dfs)

# optimization_dfs.to_csv("discount_factors.csv")

In [ ]:
optimization_dfs = pd.read_csv("discount_factors.csv")
optimization_dfs["session"]=optimization_dfs["session"].fillna("")

optimization_dfs["alpha"] = optimization_dfs["session"].apply(lambda session: alpha_val_dict[session])
optimization_dfs["discount_factor"] = optimization_dfs["session"].apply(lambda session: experiment_val_dict[session])

In [ ]:
# optimization_dfs = optimization_dfs[optimization_dfs["alpha"]==1]

In [ ]:
optimization_dfs=optimization_dfs.reset_index()
best_param_rows = optimization_dfs.loc[
    optimization_dfs.groupby(["trace_pid"]).idxmax()["value"]
]

In [ ]:
best_param_rows.groupby(["session"]).count().sort_values(by="trace_pid")

In [17]:
Counter(best_param_rows["alpha"])

Counter({1.0: 98, 0.25: 7, 0.5: 7, 0.9: 8, 0.75: 2})

In [18]:
np.mean(best_param_rows["discount_factor"]), np.std(best_param_rows["discount_factor"])

(0.7487704918032786, 0.3079854634339799)

In [19]:
Counter(best_param_rows["discount_factor"])

Counter({1.0: 60, 0.75: 29, 0.1: 11, 0.3: 3, 0.5: 8, 0.4: 7, 0.2: 4})

In [20]:
np.mean(best_param_rows["temp"]), np.std(best_param_rows["temp"])

(1.2270491803278687, 1.022515933199512)

# Simulations

In [21]:
print(f"for experiment_setting in high_increasing {' '.join(['high_increasing' + str(discount_factor) for discount_factor in discount_factors])};\n"
      f"    do for alpha in 1 {' '.join([str(alpha) for alpha in alphas])};\n"
      f"        do condor_submit_bid 2 submission_scripts/MPI-IS/06_Simulate_Optimal.sub experiment_setting=$experiment_setting cost_function=back_dist_depth_eff_forw param_file=null alpha=$alpha;\n"
      f"done; done;")

for experiment_setting in high_increasing high_increasing0.1 high_increasing0.2 high_increasing0.3 high_increasing0.4 high_increasing0.5 high_increasing0.75;
    do for alpha in 1 0.0 0.25 0.5 0.75 0.9;
        do condor_submit_bid 2 submission_scripts/MPI-IS/06_Simulate_Optimal.sub experiment_setting=$experiment_setting cost_function=back_dist_depth_eff_forw param_file=null alpha=$alpha;
done; done;


```
rsync -aPzr --include "high_increasing*/OptimalQ/simulated_agents_back_dist_depth_eff_forw*" --include "*/" --exclude "*" vfelso@login.cluster.is.localnet:/work/vfelso/planning-depth-differences/cluster/data/trajectories/ cluster/data/trajectories
```


# Look at simulations

In [22]:
depth_data = pd.concat([pd.read_csv(f, index_col=0) for f in Path("../../../cluster/data/trajectories/high_increasing/OptimalQ/").glob("simulated_agents_linear_depth_*_91.00.csv")])
depth_only_data = pd.concat([pd.read_csv(f, index_col=0) for f in Path("../../../cluster/data/trajectories/high_increasing/OptimalQ/").glob("simulated_agents_depth_only_*_91.00.csv")])

In [23]:
depth_data.groupby(["sim_depth_cost_weight"]).mean()

/var/folders/y7/qnlh5f453nxbc7w72qw0lw340000gn/T/ipykernel_15849/1404556213.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  depth_data.groupby(["sim_depth_cost_weight"]).mean()


,n_steps,i_episode,finished,return,trial_id,actions,rewards,sim_exact,sim_temperature_file,sim_num_simulated,sim_num_trials,sim_seed,sim_static_cost_weight,pid
sim_depth_cost_weight,,,,,,,,,,,,,,
-5.0,13.696003,9.501049,1.0,170.873703,1.146243e+17,7.058490,13.371199,0.0,NaN,54.221865,20.0,91.0,1.063876,26.612189
-2.5,13.517600,9.515829,1.0,107.983296,7.307269e+16,7.099310,8.630997,0.0,NaN,56.560678,20.0,91.0,0.728505,27.776230
-1.0,12.684407,9.509322,1.0,70.870104,1.363922e+17,7.279407,6.159043,0.0,NaN,55.684120,20.0,91.0,0.547208,27.326802
-0.1,11.905653,9.476758,1.0,57.395145,4.704697e+17,7.508223,5.878279,0.0,NaN,10.000000,20.0,91.0,-0.022484,4.502905
0.0,9.723910,9.529295,1.0,50.710475,4.196647e+17,8.005750,7.395792,0.0,NaN,39.757557,20.0,91.0,0.271548,19.440438
0.1,10.215781,9.452049,1.0,54.658385,4.584939e+17,7.869466,7.012584,0.0,NaN,10.000000,20.0,91.0,-0.023157,4.502474
0.5,9.653321,9.517381,1.0,46.814723,4.056651e+17,7.974976,6.552327,0.0,NaN,10.000000,20.0,91.0,0.030522,4.473556
1.0,7.842421,9.490656,1.0,34.882873,4.328797e+17,8.421235,6.828057,0.0,NaN,46.116743,20.0,91.0,0.342109,22.622292
2.5,6.477913,9.535244,1.0,13.882179,1.760264e+17,8.746095,4.689004,0.0,NaN,51.102402,20.0,91.0,0.559582,25.214378


In [24]:
depth_only_data.groupby(["sim_depth_cost_weight"]).mean()

/var/folders/y7/qnlh5f453nxbc7w72qw0lw340000gn/T/ipykernel_15849/2914157578.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  depth_only_data.groupby(["sim_depth_cost_weight"]).mean()


,n_steps,i_episode,finished,return,trial_id,actions,rewards,sim_exact,sim_temperature_file,sim_cost_parameter_values,sim_num_simulated,sim_num_trials,sim_seed,pid
sim_depth_cost_weight,,,,,,,,,,,,,,
-5.0,11.219890,9.495930,1.0,113.074833,1.788010e+17,7.551589,11.095626,0.0,NaN,-5.0,100.0,20.0,91.0,49.564535
-2.5,11.240762,9.498116,1.0,75.289536,1.169812e+17,7.545935,7.376487,0.0,NaN,-2.5,100.0,20.0,91.0,49.522197
-1.0,10.221193,9.532738,1.0,52.700149,2.201080e+17,7.718683,5.806498,0.0,NaN,-1.0,100.0,20.0,91.0,49.693309
0.0,6.278772,9.443878,1.0,38.413891,2.443176e+17,8.876141,9.982103,0.0,NaN,0.0,100.0,20.0,91.0,49.568211
1.0,6.266260,9.451893,1.0,29.799602,6.096593e+17,8.870972,8.274698,0.0,NaN,1.0,100.0,20.0,91.0,49.681706
2.5,6.054337,9.507554,1.0,19.120614,2.966608e+17,8.935916,6.570967,0.0,NaN,2.5,100.0,20.0,91.0,49.063962
5.0,6.117969,9.534557,1.0,0.403003,3.319853e+17,8.849500,2.752860,0.0,NaN,5.0,100.0,20.0,91.0,49.481173
7.5,4.199356,9.428418,1.0,0.105941,3.002324e+16,9.760558,2.525412,0.0,NaN,7.5,100.0,20.0,91.0,49.544381
10.0,4.219843,9.468772,1.0,-11.576017,5.926656e+16,9.750892,-0.914347,0.0,NaN,10.0,100.0,20.0,91.0,49.325125


In [25]:
discount_data = []
for discount_factor, experiment_suffix in zip([1] + discount_factors, [""] + discount_factors):
    for alpha, alpha_suffix in zip([1]+alphas, [""] + [f"_{alpha}" for alpha in alphas]):
        curr_df = pd.read_csv(f"../../../cluster/data/trajectories/high_increasing{experiment_suffix}{alpha_suffix}/OptimalQ/simulated_agents_back_dist_depth_eff_forw_0.00_0.00_0.00_0.00_1.00_91.00.csv", index_col=0) 
        curr_df["discount_factor"] = discount_factor
        curr_df["alpha"] = alpha
        discount_data.append(curr_df)
discount_data = pd.concat(discount_data)

In [26]:
discount_data.groupby(["discount_factor", "alpha"]).mean()

/var/folders/y7/qnlh5f453nxbc7w72qw0lw340000gn/T/ipykernel_15849/3836188975.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  discount_data.groupby(["discount_factor", "alpha"]).mean()


n_steps  i_episode  finished     return      trial_id  \
discount_factor alpha                                                           
0.10            0.00   3.619048   9.482143       1.0   0.503968 -1.882007e+17   
                0.25   3.642436   9.351670       1.0   0.440079 -9.922029e+16   
                0.50   3.570850   9.246964       1.0   0.676113  6.087526e+16   
                0.75   3.604790   9.636727       1.0   0.335329  5.862370e+17   
                0.90   3.575758   9.579798       1.0   0.791919  6.288288e+17   
                1.00   3.628458   9.452569       1.0   0.498024 -5.848669e+17   
0.20            0.00   4.294627   9.639515       1.0  -0.407626  2.050235e+16   
                0.25   4.007273   9.120000       1.0   0.201455  1.729650e+16   
                0.50   3.981584   9.425414       1.0   0.185635 -7.240391e+17   
                0.75   4.624473   9.585091       1.0   0.037918 -8.954899e+16   
                0.90   4.516691   9.455733       1.0   0.551408  4.814089e+16   
                1.00   5.038961   9.562338       1.0   0.310338  3.753779e+16   
0.30            0.00   4.462585   9.777211       1.0   0.799456  5.023438e+17   
                0.25   4.059041   9.562731       1.0   1.125166  5.157011e+17   
                0.50   4.137931   9.653358       1.0   1.033466  2.717881e+17   
                0.75   4.181159   9.461957       1.0   1.109710 -4.513491e+17   
                0.90   4.437396   9.714524       1.0   0.559199 -5.868607e+16   
                1.00   6.186263   9.473807       1.0  -0.014994  3.344245e+17   
0.40            0.00   4.169118   9.336397       1.0   3.612059  1.413899e+17   
                0.25   6.028986   9.277778       1.0   3.022512 -4.346857e+16   
                0.50   6.103529   9.610588       1.0   2.950494  2.308041e+17   
                0.75   5.964029   9.496403       1.0   2.995492  1.011637e+18   
                0.90   5.757419   9.336774       1.0   3.420490 -6.507025e+16   
                1.00   6.160096   9.788530       1.0   2.704898  8.304249e+17   
0.50            0.00   5.902622   9.459426       1.0   6.838951  7.137023e+17   
                0.25   6.149758   9.486715       1.0   6.666667  3.500624e+17   
                0.50   6.196643   9.574341       1.0   6.335731  1.182485e+18   
                0.75   6.407240   9.719457       1.0   5.977376  3.205938e+17   
                0.90   5.823529   9.453964       1.0   7.107417  8.460159e+16   
                1.00   6.670442   9.383918       1.0   5.531144  7.776593e+17   
0.75            0.00   5.959900   9.462406       1.0  20.211153 -6.419895e+17   
                0.25   6.044496   9.416862       1.0  20.334895  5.143208e+17   
                0.50   6.024242   9.760000       1.0  20.501212  5.261668e+17   
                0.75   6.004963   9.617866       1.0  20.354218  6.202707e+17   
                0.90   5.908163   9.358418       1.0  20.920918 -3.330978e+17   
                1.00   6.506271   9.627138       1.0  19.383124  7.733678e+17   
1.00            0.00   6.364679   9.459862       1.0  37.887615  5.045432e+17   
                0.25   6.399067   8.884481       1.0  37.024504  1.157205e+18   
                0.50   6.518602   9.545660       1.0  36.919955  1.971777e+17   
                0.75   6.399530   9.598120       1.0  36.770858  4.011088e+17   
                0.90   6.042660   9.340025       1.0  38.878294  1.242264e+17   
                1.00   6.310427   9.665877       1.0  37.808057  6.684998e+17   

                        actions    rewards  sim_exact  sim_temperature_file  \
discount_factor alpha                                                         
0.10            0.00   8.444444   0.301587        0.0                   NaN   
                0.25   8.324165   0.269155        0.0                   NaN   
                0.50   8.514170   0.380567        0.0                   NaN   
                0.75   8.433134   0.241517        0.0           

In [27]:
sim_cost_parameters={}
for cost_function in ["linear_depth", "back_dist_depth_eff_forw"]:
    yaml_file = Path(
                        f"../../../data/inputs/yamls/cost_functions/{cost_function}.yaml"
                    )
    with open(str(yaml_file), "r") as stream:
        cost_details = yaml.safe_load(stream)

    sim_cost_parameters[cost_function] = [
                                        f"sim_{param}" for param in cost_details["constant_values"].keys()
                                        ]

In [28]:
for classification, nodes in experiment_setting_data[
    "node_classification"
].items():
    discount_data[classification] = discount_data["actions"].apply(
        lambda action: action in nodes
    )
    depth_only_data[classification] = depth_only_data["actions"].apply(
        lambda action: action in nodes
    )
    depth_data[classification] = depth_data["actions"].apply(
        lambda action: action in nodes
    )

sum_discount_clicks = (
    discount_data.groupby(["pid", "i_episode", *sim_cost_parameters["back_dist_depth_eff_forw"], "discount_factor", "alpha"])
    .sum()
    .reset_index()
    .groupby([*sim_cost_parameters["back_dist_depth_eff_forw"], "discount_factor", "alpha"])
    .mean()
    .reset_index()
)

sum_depth_clicks = (
    depth_data.groupby(["pid", "i_episode", *sim_cost_parameters["linear_depth"]])
    .sum()
    .reset_index()
    .groupby([ *sim_cost_parameters["linear_depth"]])
    .mean()
    .reset_index()
)

NameError: name 'experiment_setting_data' is not defined

In [ ]:
sum_depth_clicks.groupby([ "sim_depth_cost_weight"]).mean()[["early", "middle", "late", "clicks"]]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sum_df=sum_discount_clicks.groupby([ "discount_factor", "alpha"]).mean().reset_index()

for field in ["early", "middle", "late", "clicks"]:
    heat_map_data = sum_df.pivot(index="discount_factor", columns="alpha", values=field)
    plt.figure()
    plt.title(field)
    sns.heatmap(data=heat_map_data, annot=True, fmt=".2f")
